# Day 15 - o1

In [1]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def parse_map_and_moves(lines):
    # The map is typically a block of lines followed by move lines.
    # The map should be enclosed by '#' lines. We will assume that the first line starts with '#' and
    # the map continues until we hit a line that cannot be map (i.e., does not look like part of the map).
    # After that, all remaining lines are moves (concatenated).
    
    # Identify where the map ends. The map should at least have a top line of '#'.
    # We'll find the largest rectangular block of rows that look like part of the map
    # and then assume the rest are moves.
    # A heuristic: The map lines all have the same length and are surrounded by '#' at top and bottom.
    
    # First, let's figure out the dimensions. We'll assume the map lines all start and end with '#'
    # and we have at least two lines of map.
    
    # Find the first line of moves. Moves contain only ^, v, <, > (potentially plus empty lines)
    # Let's try reading until we find a line that doesn't resemble map.
    
    # A map line should have '#' characters at least at start and end. The puzzle states the map is rectangular.
    # We'll guess the map lines are continuous from the start until we reach a line that doesn't match the length.
    
    # We'll first determine the length of the first line, which should be a full '#' line.
    # Then we read subsequent lines as map lines until we hit something that doesn't match the length or is empty.
    
    map_lines = []
    i = 0
    # The first line should be a wall line (like '##########')
    # We'll just read lines until we hit a line that doesn't contain '#' '.' 'O' '@' or is empty.
    # If a line has other characters, or is shorter/longer unexpectedly, we consider that map ended.
    
    # Find the width from the first line
    width = len(lines[0])
    
    for line in lines:
        if len(line) == width and all(ch in '#.@O.' for ch in line):
            map_lines.append(line)
            i += 1
        else:
            break
    
    # The rest of the lines are moves (could be multiple lines)
    moves_lines = lines[i:]
    moves_str = "".join(moves_lines)  # concatenate
    # Filter out only the move characters
    moves_str = "".join(ch for ch in moves_str if ch in '^v<>')
    
    return map_lines, moves_str

def simulate(map_lines, moves_str):
    # Convert map_lines into a 2D list for mutability
    grid = [list(row) for row in map_lines]
    rows = len(grid)
    cols = len(grid[0])
    
    # Find the robot position '@'
    robot_r = robot_c = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '@':
                robot_r, robot_c = r, c
                break
        if robot_r is not None:
            break
    
    # Directions
    deltas = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    def can_push(r, c, dr, dc):
        # Tries to push boxes starting at position (r,c) in direction (dr,dc)
        # If cell (r,c) is not a box, pushing is trivial (no push needed).
        # But if it is a box, we need to push it and all boxes behind it recursively.
        
        # If out of bounds or hits a wall, can't move
        nr, nc = r + dr, c + dc
        if grid[r][c] == '#':
            return False
        
        if grid[r][c] == 'O':
            # Need to push this box forward
            if grid[nr][nc] == '#':
                return False
            if grid[nr][nc] == 'O':
                # Need to push the next box as well
                if not can_push(nr, nc, dr, dc):
                    return False
            elif grid[nr][nc] not in ('.', '@'):
                # It's something else (like a wall or out of bounds)
                return False
        # If pushing is possible, do it (after confirming)
        return True
    
    def do_push(r, c, dr, dc):
        # Actually perform the push, assuming can_push succeeded
        nr, nc = r + dr, c + dc
        if grid[r][c] == 'O':
            # Push the next box first (to not overwrite)
            if grid[nr][nc] == 'O':
                do_push(nr, nc, dr, dc)
            # Move this box forward
            grid[nr][nc] = 'O'
            grid[r][c] = '.'
    
    for move in moves_str:
        dr, dc = deltas[move]
        nr, nc = robot_r + dr, robot_c + dc
        
        # Check if move is possible
        if grid[nr][nc] == '#':
            # Can't move into a wall
            continue
        elif grid[nr][nc] == '.':
            # Move is free
            grid[robot_r][robot_c], grid[nr][nc] = '.', '@'
            robot_r, robot_c = nr, nc
        elif grid[nr][nc] == 'O':
            # Need to push
            # Check if we can push the chain of boxes starting from (nr,nc)
            if can_push(nr, nc, dr, dc):
                # Perform push
                do_push(nr, nc, dr, dc)
                # Move robot
                grid[robot_r][robot_c], grid[nr][nc] = '.', '@'
                robot_r, robot_c = nr, nc
            else:
                # Can't push, do nothing
                continue
        elif grid[nr][nc] == '@':
            # Shouldn't happen since there's only one robot
            continue
        else:
            # Unknown tile (shouldn't happen)
            continue
    
    # After all moves, compute sum of GPS coordinates of boxes
    total = 0
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'O':
                # GPS coordinate = r*100 + c
                total += r*100 + c
    
    return total

def main():
    lines = read_input('input.txt')
    map_lines, moves_str = parse_map_and_moves(lines)
    result = simulate(map_lines, moves_str)
    print(result)

if __name__ == '__main__':
    main()


1436690


## Part 2

In [ ]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def parse_map_and_moves(lines):
    # Similar approach as part one:
    # The map lines come first. We'll assume all map lines have the same length and are valid.
    # After the map lines, we get the move lines, which must be concatenated and filtered.
    map_lines = []
    i = 0
    width = len(lines[0])
    for line in lines:
        if len(line) == width and all(ch in '#.@O.' for ch in line):
            map_lines.append(line)
            i += 1
        else:
            break
    
    moves_lines = lines[i:]
    moves_str = "".join(moves_lines)
    moves_str = "".join(ch for ch in moves_str if ch in '^v<>')
    return map_lines, moves_str

def scale_map(map_lines):
    # Scale the map horizontally as per instructions:
    # '#' -> '##'
    # 'O' -> '[]'
    # '.' -> '..'
    # '@' -> '@.'
    # Every line doubles in length, but logically we still consider the same number of cells.
    translation = {
        '#': '##',
        'O': '[]',
        '.': '..',
        '@': '@.'
    }
    
    scaled = []
    for line in map_lines:
        new_line = []
        for ch in line:
            new_line.append(translation[ch])
        scaled.append("".join(new_line))
    return scaled

def simulate(map_lines, moves_str):
    # Convert scaled map into a 2D grid of cells,
    # where each cell is represented by 2 characters.
    # We'll keep a grid of strings, each string is 2 chars.
    rows = len(map_lines)
    line_length = len(map_lines[0])
    # original number of columns = line_length // 2
    cols = line_length // 2
    
    # Build cell-based grid
    grid = []
    for r in range(rows):
        row_cells = []
        line = map_lines[r]
        for c in range(cols):
            cell = line[2*c:2*c+2]
            row_cells.append(cell)
        grid.append(row_cells)
    
    # Find the robot position '@.'
    robot_r = robot_c = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c].startswith('@'):
                robot_r, robot_c = r, c
                break
        if robot_r is not None:
            break
    
    deltas = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    def in_bounds(r, c):
        return 0 <= r < rows and 0 <= c < cols
    
    def can_push(r, c, dr, dc):
        # If the next cell is out of bounds or a wall, can't push.
        nr, nc = r + dr, c + dc
        if not in_bounds(nr, nc):
            return False
        if grid[nr][nc].startswith('#'):
            return False
        
        # If this cell is a box, we need to push it forward as well.
        if grid[r][c].startswith('['):  # '[]' means box
            # If next cell is also a box, recurse.
            if grid[nr][nc].startswith('['):
                if not can_push(nr, nc, dr, dc):
                    return False
            elif not (grid[nr][nc].startswith('.') or grid[nr][nc].startswith('@')):
                # next cell is neither empty nor robot
                return False
        
        return True
    
    def do_push(r, c, dr, dc):
        # Perform the push, assuming can_push passed
        nr, nc = r + dr, c + dc
        if grid[r][c].startswith('['):
            # Push the next box first to avoid overwriting
            if grid[nr][nc].startswith('['):
                do_push(nr, nc, dr, dc)
            # Move this box forward
            grid[nr][nc] = '[]'
            grid[r][c] = '..'
    
    for move in moves_str:
        dr, dc = deltas[move]
        nr, nc = robot_r + dr, robot_c + dc
        
        # Check bounds
        if not in_bounds(nr, nc):
            # can't move outside map
            continue
        
        # Check cell we want to move into
        target = grid[nr][nc]
        if target.startswith('#'):
            # can't move into wall
            continue
        elif target.startswith('.'):
            # free move
            grid[robot_r][robot_c] = '..'  # leave empty
            grid[nr][nc] = '@.'
            robot_r, robot_c = nr, nc
        elif target.startswith('['):
            # box push attempt
            if can_push(nr, nc, dr, dc):
                do_push(nr, nc, dr, dc)
                # move robot
                grid[robot_r][robot_c] = '..'
                grid[nr][nc] = '@.'
                robot_r, robot_c = nr, nc
            else:
                # can't push
                continue
        elif target.startswith('@'):
            # shouldn't happen, only one robot
            continue
        else:
            # unknown cell type?
            continue
    
    # Compute sum of GPS coordinates for boxes
    # For each box '[]', coordinate = 100*r + c
    total = 0
    for r in range(rows):
        for c in range(cols):
            if grid[r][c].startswith('['):
                total += 100*r + c
    return total

def main():
    lines = read_input('input.txt')
    map_lines, moves_str = parse_map_and_moves(lines)
    scaled_map = scale_map(map_lines)
    result = simulate(scaled_map, moves_str)
    print(result)

if __name__ == '__main__':
    main()


1436690


In [5]:
def simulate(map_lines, moves_str):
    rows = len(map_lines)
    line_length = len(map_lines[0])
    # Each cell is 2 chars wide, so number of cells per row:
    cols = line_length // 2

    # Convert the map into a grid of cells (each cell is 2 chars)
    grid = []
    for r in range(rows):
        row_cells = []
        line = map_lines[r]
        for c in range(cols):
            cell = line[2*c:2*c+2]
            row_cells.append(cell)
        grid.append(row_cells)

    # Find the robot
    robot_r = robot_c = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c].startswith('@'):
                robot_r, robot_c = r, c
                break
        if robot_r is not None:
            break

    deltas = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }

    def in_bounds(r, c):
        return 0 <= r < rows and 0 <= c < cols

    def can_push(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        if not in_bounds(nr, nc):
            return False
        if grid[nr][nc].startswith('#'):
            return False
        if grid[r][c].startswith('['):
            # pushing another box
            if grid[nr][nc].startswith('['):
                if not can_push(nr, nc, dr, dc):
                    return False
        return True

    def do_push(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        if grid[r][c].startswith('['):
            if grid[nr][nc].startswith('['):
                do_push(nr, nc, dr, dc)
            grid[nr][nc] = '[]'
            grid[r][c] = '..'

    for move in moves_str:
        dr, dc = deltas[move]
        nr, nc = robot_r + dr, robot_c + dc
        if not in_bounds(nr, nc):
            continue
        target = grid[nr][nc]
        if target.startswith('#'):
            continue
        elif target.startswith('.'):
            # free move
            grid[robot_r][robot_c] = '..'
            grid[nr][nc] = '@.'
            robot_r, robot_c = nr, nc
        elif target.startswith('['):
            # box push
            if can_push(nr, nc, dr, dc):
                do_push(nr, nc, dr, dc)
                grid[robot_r][robot_c] = '..'
                grid[nr][nc] = '@.'
                robot_r, robot_c = nr, nc
            else:
                continue

    # Compute sum of GPS coordinates
    # Now use 2*c for the column part
    total = 0
    for r in range(rows):
        for c in range(cols):
            if grid[r][c].startswith('['):
                total += 100 * r + (2 * c)

    return total

def main():
    lines = read_input('input.txt')
    map_lines, moves_str = parse_map_and_moves(lines)
    scaled_map = scale_map(map_lines)
    result = simulate(scaled_map, moves_str)
    print(result)

if __name__ == '__main__':
    main()



1451880


In [6]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def parse_map_and_moves(lines):
    # Identify map lines and moves
    map_lines = []
    width = len(lines[0])
    i = 0
    for line in lines:
        if len(line) == width and all(ch in '#.@O.' for ch in line):
            map_lines.append(line)
            i += 1
        else:
            break

    moves_lines = lines[i:]
    moves_str = "".join(moves_lines)
    moves_str = "".join(ch for ch in moves_str if ch in '^v<>')
    return map_lines, moves_str

def scale_map(map_lines):
    # Scale the map horizontally
    translation = {
        '#': '##',
        'O': '[]',
        '.': '..',
        '@': '@.'
    }

    scaled = []
    for line in map_lines:
        new_line = []
        for ch in line:
            new_line.append(translation[ch])
        scaled.append("".join(new_line))
    return scaled

def simulate(map_lines, moves_str):
    rows = len(map_lines)
    line_length = len(map_lines[0])
    # Each cell now is 2 chars wide
    cols = line_length // 2

    # Build a grid of cells (each cell is 2 characters)
    grid = []
    for r in range(rows):
        row_cells = []
        line = map_lines[r]
        for c in range(cols):
            cell = line[2*c:2*c+2]
            row_cells.append(cell)
        grid.append(row_cells)

    # Find the robot '@.'
    robot_r = robot_c = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c].startswith('@'):
                robot_r, robot_c = r, c
                break
        if robot_r is not None:
            break

    deltas = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }

    def in_bounds(r, c):
        return 0 <= r < rows and 0 <= c < cols

    def can_push(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        if not in_bounds(nr, nc):
            return False
        if grid[nr][nc].startswith('#'):
            return False
        if grid[r][c].startswith('['):  # pushing a box
            if grid[nr][nc].startswith('['):
                # Need to push the next box as well
                if not can_push(nr, nc, dr, dc):
                    return False
        return True

    def do_push(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        if grid[r][c].startswith('['):
            if grid[nr][nc].startswith('['):
                do_push(nr, nc, dr, dc)
            grid[nr][nc] = '[]'
            grid[r][c] = '..'

    # Process moves
    for move in moves_str:
        dr, dc = deltas[move]
        nr, nc = robot_r + dr, robot_c + dc
        if not in_bounds(nr, nc):
            continue
        target = grid[nr][nc]
        if target.startswith('#'):
            continue
        elif target.startswith('.'):
            # Move robot freely
            grid[robot_r][robot_c] = '..'
            grid[nr][nc] = '@.'
            robot_r, robot_c = nr, nc
        elif target.startswith('['):
            # Need to push
            if can_push(nr, nc, dr, dc):
                do_push(nr, nc, dr, dc)
                grid[robot_r][robot_c] = '..'
                grid[nr][nc] = '@.'
                robot_r, robot_c = nr, nc
            else:
                # Can't push
                continue

    # After simulation, reconstruct final lines
    final_map_lines = []
    for r in range(rows):
        line = "".join(grid[r])
        final_map_lines.append(line)

    # Compute sum of GPS coordinates of boxes
    # For each box '[]', find the position of '['
    # GPS = 100 * row + (character_index_of_[)
    total = 0
    for r, line in enumerate(final_map_lines):
        start = 0
        while True:
            idx = line.find('[', start)
            if idx == -1:
                break
            # idx is the character position of '[' from the left edge (0-based)
            total += 100*r + idx
            start = idx + 1

    return total

def main():
    lines = read_input('input.txt')
    original_map, moves_str = parse_map_and_moves(lines)
    scaled_map = scale_map(original_map)
    result = simulate(scaled_map, moves_str)
    print(result)

if __name__ == '__main__':
    main()


1451880


In [7]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def parse_map_and_moves(lines):
    # Identify map lines and moves
    map_lines = []
    width = len(lines[0])
    i = 0
    for line in lines:
        if len(line) == width and all(ch in '#.@O.' for ch in line):
            map_lines.append(line)
            i += 1
        else:
            break

    moves_lines = lines[i:]
    moves_str = "".join(moves_lines)
    moves_str = "".join(ch for ch in moves_str if ch in '^v<>')
    return map_lines, moves_str

def scale_map(map_lines):
    # Scale the map horizontally as per instructions
    translation = {
        '#': '##',
        'O': '[]',
        '.': '..',
        '@': '@.'
    }

    scaled = []
    for line in map_lines:
        new_line = []
        for ch in line:
            new_line.append(translation[ch])
        scaled.append("".join(new_line))
    return scaled

def simulate(map_lines, moves_str):
    rows = len(map_lines)
    line_length = len(map_lines[0])
    # Each cell is now 2 characters wide
    cols = line_length // 2

    # Convert the scaled map into a grid of cells
    grid = []
    for r in range(rows):
        row_cells = []
        line = map_lines[r]
        for c in range(cols):
            cell = line[2*c:2*c+2]
            row_cells.append(cell)
        grid.append(row_cells)

    # Find the robot position '@.'
    robot_r = robot_c = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c].startswith('@'):
                robot_r, robot_c = r, c
                break
        if robot_r is not None:
            break

    deltas = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }

    def in_bounds(r, c):
        return 0 <= r < rows and 0 <= c < cols

    def can_push(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        if not in_bounds(nr, nc):
            return False
        if grid[nr][nc].startswith('#'):
            return False
        if grid[r][c].startswith('['):  # box
            if grid[nr][nc].startswith('['):
                # Need to push further
                if not can_push(nr, nc, dr, dc):
                    return False
        return True

    def do_push(r, c, dr, dc):
        nr, nc = r + dr, c + dc
        if grid[r][c].startswith('['):
            if grid[nr][nc].startswith('['):
                do_push(nr, nc, dr, dc)
            grid[nr][nc] = '[]'
            grid[r][c] = '..'

    # Process each move
    for move in moves_str:
        dr, dc = deltas[move]
        nr, nc = robot_r + dr, robot_c + dc
        if not in_bounds(nr, nc):
            continue
        target = grid[nr][nc]
        if target.startswith('#'):
            continue
        elif target.startswith('.'):
            # Move robot
            grid[robot_r][robot_c] = '..'
            grid[nr][nc] = '@.'
            robot_r, robot_c = nr, nc
        elif target.startswith('['):
            # Need to push boxes
            if can_push(nr, nc, dr, dc):
                do_push(nr, nc, dr, dc)
                grid[robot_r][robot_c] = '..'
                grid[nr][nc] = '@.'
                robot_r, robot_c = nr, nc
            else:
                # Can't push
                continue

    # After all moves, reconstruct the full lines
    final_map_lines = []
    for r in range(rows):
        line = "".join(grid[r])
        final_map_lines.append(line)

    # Now calculate GPS coordinates based on character positions of '['
    total = 0
    for r, line in enumerate(final_map_lines):
        start = 0
        while True:
            idx = line.find('[', start)
            if idx == -1:
                break
            # GPS = 100 * r + idx (since idx is the character position of the '[')
            total += 100*r + idx
            start = idx + 1

    return total

def main():
    lines = read_input('input.txt')
    original_map, moves_str = parse_map_and_moves(lines)
    scaled_map = scale_map(original_map)
    result = simulate(scaled_map, moves_str)
    print(result)

if __name__ == '__main__':
    main()


1451880


In [8]:
def read_input(filename):
    with open(filename, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return lines

def parse_map_and_moves(lines):
    # Identify map lines and move lines (same logic as Part One)
    width = len(lines[0])
    map_lines = []
    i = 0
    for line in lines:
        if len(line) == width and all(ch in '#.@O.' for ch in line):
            map_lines.append(line)
            i += 1
        else:
            break
    moves_lines = lines[i:]
    moves_str = "".join(moves_lines)
    moves_str = "".join(ch for ch in moves_str if ch in '^v<>')
    return map_lines, moves_str

def scale_map(map_lines):
    # Scale the map horizontally:
    # '#' -> '##'
    # 'O' -> '[]'
    # '.' -> '..'
    # '@' -> '@.'
    scaled_map = []
    for line in map_lines:
        new_line = []
        for ch in line:
            if ch == '#':
                new_line.append('##')
            elif ch == 'O':
                new_line.append('[]')
            elif ch == '.':
                new_line.append('..')
            elif ch == '@':
                new_line.append('@.')
            else:
                # Unexpected character
                new_line.append('..')  # safe fallback
        scaled_map.append("".join(new_line))
    return scaled_map

def simulate(scaled_map, moves_str):
    # Now each tile is 2 chars wide.
    # Find the robot tile '@.' and record its tile-position (r,c).
    # Each row is a string, each tile is 2 chars.
    rows = len(scaled_map)
    cols_in_chars = len(scaled_map[0])  # total chars in a row
    # Number of tiles horizontally is cols_in_chars // 2
    tile_cols = cols_in_chars // 2

    grid = [list(row) for row in scaled_map]

    robot_r = robot_c = None
    for r in range(rows):
        row_str = grid[r]
        for c in range(tile_cols):
            if row_str[2*c] == '@':
                robot_r, robot_c = r, c
                break
        if robot_r is not None:
            break

    deltas = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }

    def tile(r, c):
        # returns the 2-char tile at (r,c)
        return "".join(grid[r][2*c:2*c+2])

    def set_tile(r, c, val):
        # val is a 2-char string
        for i, ch in enumerate(val):
            grid[r][2*c+i] = ch

    def can_push(r, c, dr, dc):
        # Check if we can push boxes starting from tile (r,c) forward
        # If tile is a box '[]', need to push forward
        # If tile is '.' or '@' or '#', handle accordingly
        if tile(r, c) == '[]':
            nr, nc = r+dr, c+dc
            nxt = tile(nr, nc)
            if nxt == '##':  # wall
                return False
            if nxt == '[]':  # another box, must push it first
                if not can_push(nr, nc, dr, dc):
                    return False
            elif nxt not in ('..', '@.', '[]', '##'):
                # Unexpected tile type
                return False
        # If it's not a box, no special pushing needed.
        return True

    def do_push(r, c, dr, dc):
        # Actually perform the push, assuming can_push succeeded
        if tile(r, c) == '[]':
            nr, nc = r+dr, c+dc
            if tile(nr, nc) == '[]':
                do_push(nr, nc, dr, dc)
            # move this box forward
            set_tile(nr, nc, '[]')
            set_tile(r, c, '..')

    for move in moves_str:
        dr, dc = deltas[move]
        nr, nc = robot_r+dr, robot_c+dc
        # Check boundaries
        if not (0 <= nr < rows and 0 <= nc < tile_cols):
            # Out of bounds - can't move
            continue
        ntile = tile(nr, nc)
        if ntile == '##':
            # wall
            continue
        elif ntile == '..':
            # move robot
            set_tile(robot_r, robot_c, '..')
            set_tile(nr, nc, '@.')
            robot_r, robot_c = nr, nc
        elif ntile == '[]':
            # push
            if can_push(nr, nc, dr, dc):
                do_push(nr, nc, dr, dc)
                # move robot
                set_tile(robot_r, robot_c, '..')
                set_tile(nr, nc, '@.')
                robot_r, robot_c = nr, nc
            else:
                # can't push
                continue
        elif ntile == '@.':
            # robot tile again? Shouldn't happen. Just skip.
            continue
        else:
            # Unknown tile type
            continue

    # After all moves, compute sum of boxes' GPS coordinates.
    # For each '[' char in the final grid, add (r*100 + c) where c is the char-column of '['.
    total = 0
    for r in range(rows):
        for c in range(len(grid[r])):
            if grid[r][c] == '[':
                total += r*100 + c
    return total

def main():
    lines = read_input('input.txt')
    map_lines, moves_str = parse_map_and_moves(lines)
    scaled = scale_map(map_lines)
    result = simulate(scaled, moves_str)
    print(result)

if __name__ == '__main__':
    main()


1451880
